In [1]:
from chunk_pipeline import *
from prediction_ML_pipeline import save_dataframe_to_folder
import time
import os

logging.getLogger().setLevel(logging.WARNING)

## ---------ADJUST THIS---------------

ticker_lst = ['AES']


skip_training = True
year = "2018"
specific_date = "2018-01-30"
order_type_lst = ['combined']
predictive_lst = [True]
ret_type_lst = ["log_ret_ex"]
momentum_lst = [True]
file_name_lst= ["NOT_SAVING.csv"]

delta_lst = ['15S', '30S', '1min', '2min', '5min', '10min', '15min', '30min', '60min']

## ------------------------------------

# Check if files exist
full_results_df = None
folder_path = "/nfs/home/jingt/dissertation-iceberg/data/output_results/2018"
model_path = "/nfs/home/jingt/dissertation-iceberg/data/output_folder"
results_df_all = {i: [] for i in file_name_lst}

for ticker in ticker_lst:

    file_path_train = f'/nfs/home/jingt/dissertation-iceberg/data/training_data/_data_dwn_32_210_hidden_liquidity_{ticker}_2012-01-01_2012-12-31_1.7z'
    file_path_test = f"/nfs/data/lobster_data/lobster_raw/2017-19/_data_dwn_32_302__{ticker}_{year}-01-01_{year}-12-31_10.7z"

    if not os.path.exists(file_path_train):
        print(f"The file {file_path_train} does not exist.")
    if not os.path.exists(file_path_test):
        print(f"The file {file_path_test} does not exist.")
print("All files exist!")


for ticker in ticker_lst:

    model_name = f"xgboost_{ticker}.json"

    print(f"For ticker: {ticker}")
    start = time.time()
    archive_train_path = f"/nfs/home/jingt/dissertation-iceberg/data/training_data/_data_dwn_32_210_hidden_liquidity_{ticker}_2012-01-01_2012-12-31_1.7z"
    arhive_test_path = f"/nfs/data/lobster_data/lobster_raw/2017-19/_data_dwn_32_302__{ticker}_{year}-01-01_{year}-12-31_10.7z"


    params = {
        'objective': 'binary:logistic', 
        'learning_rate': 0.1,
        'max_depth': 6,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'n_estimators': 100,
        'eval_metric': 'logloss',  # Log loss for binary classification
    }

    if not skip_training:
        print("Model fitting started", flush=True)
        model, acc = process_and_train_xgb(archive_train_path, model_path, model_name, params=params)
        print("Model fitting completed!", flush=True)
    else:
        model = None
        acc = 0

    for order_type, file_name, predictive, ret_type, momentum in zip(order_type_lst, file_name_lst,
                                                                        predictive_lst, ret_type_lst,
                                                                        momentum_lst):
        print(f"For {order_type}, pred: {predictive}, ret_type: {ret_type}, momentum: {momentum}")
        print(f"{time.time() - start:.3f} seconds elapsed", flush=True)
        start = time.time()
        print("Order imbalance calculation started", flush=True)
        df_dict = order_imbalance_calc(arhive_test_path, delta_lst=delta_lst, model=model,
                                    model_path=model_path, model_name=model_name, 
                                    order_type=order_type, specific_date=specific_date)

        print("Order imbalance calculation completed!", flush=True)
    
        print(f"{time.time() - start:.3f} seconds elapsed", flush=True)

        start = time.time()

        print("Results construction started", flush=True)
        results_df = OI_results(df_dict, order_type=order_type, predictive=predictive,
                                ret_type=ret_type, momentum=momentum)
        results_df['ticker'] = ticker
        results_df['model_acc'] = acc

        with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
            print(results_df, flush=True)

        print("Results construction completed!", flush=True)

        print(f"{time.time() - start:.3f} seconds elapsed", flush=True)

        start = time.time()

        print("Saving to folder", flush=True)

        results_df_all[file_name].append(results_df)
        full_results_df = pd.concat(results_df_all[file_name])

All files exist!
For ticker: AES
For combined, pred: True, ret_type: log_ret_ex, momentum: True
0.000 seconds elapsed
Order imbalance calculation started
Processed files: AES_2018-01-30_34200000_57600000_orderbook_10.csv AES_2018-01-30_34200000_57600000_message_10.csv
Order imbalance calculation completed!
6.015 seconds elapsed
Results construction started


2024-08-17 16:12:54,551 - ERROR - Error in final model fit: singular matrix
2024-08-17 16:12:54,552 - ERROR - Error in lm_analysis for delta 60min: not enough values to unpack (expected 4, got 2)


  timeframe  intercept    adj_R2  params_vis  tvals_vis  params_hid  \
0       15S  -0.000004 -0.000066    0.000024   0.738477    0.000011   
1       30S  -0.000012 -0.004043   -0.000011  -0.195584    0.000007   
2      1min  -0.000028 -0.004792   -0.000055  -0.576878    0.000013   
3      2min  -0.000027 -0.016917    0.000013   0.083866   -0.000012   
4      5min  -0.000005 -0.037395   -0.000030  -0.078349    0.000048   
5     10min  -0.000017 -0.097151   -0.000002  -0.002042    0.000019   
6     15min   0.000016 -0.158928   -0.000003  -0.001907   -0.000013   
7     30min  -0.000020 -0.439584    0.000002   0.000260    0.000004   

   tvals_hid  params_momentum  tvals_momentum ticker  model_acc  
0   0.339990     7.238824e-08    2.299594e-06    AES          0  
1   0.131855     1.201566e-07    2.802733e-06    AES          0  
2   0.125567    -1.512130e-07   -2.609624e-06    AES          0  
3  -0.074752    -1.230622e-08   -1.504249e-07    AES          0  
4   0.137301    -3.131112e-07 

In [2]:
df_dict['1min']

,datetime_bins,order_imbalance_vis,first_midprice,last_midprice,first_weighted_mp,last_weighted_mp,log_ret,fut_log_ret,weighted_log_ret,fut_weighted_log_ret,...,fret_tClose,daily_ret,fut_daily_ret,fret_ClOp,fret_ClCl,daily_ret_ex,fut_daily_ret_ex,fret_ClOp_ex,fret_ClCl_ex,order_imbalance_hid
0,2018-01-30 09:31:00,0.249479,117950.0,118450.0,117950.000000,118410.263022,0.004230,-0.006777,0.003895,-0.006335,...,-9.451009,-0.014536,0.000865,0.006079,0.006945,-0.01156,0.003805,0.002642,0.006448,-0.187515
1,2018-01-30 09:32:00,-0.291682,118450.0,117650.0,118410.263022,117662.500000,-0.006777,-0.004259,-0.006335,-0.004462,...,-9.444232,-0.014536,0.000865,0.006079,0.006945,-0.01156,0.003805,0.002642,0.006448,-0.836726
2,2018-01-30 09:33:00,-0.437851,117650.0,117150.0,117657.142857,117133.333333,-0.004259,0.000000,-0.004462,0.000000,...,-9.439973,-0.014536,0.000865,0.006079,0.006945,-0.01156,0.003805,0.002642,0.006448,0.556370
3,2018-01-30 09:34:00,-0.357258,117150.0,117150.0,117150.000000,117150.000000,0.000000,0.000000,0.000000,0.000000,...,-9.439973,-0.014536,0.000865,0.006079,0.006945,-0.01156,0.003805,0.002642,0.006448,0.000000
4,2018-01-30 09:35:00,-0.375000,117150.0,117150.0,117150.000000,117150.000000,0.000000,-0.001709,0.000000,-0.001908,...,-9.439973,-0.014536,0.000865,0.006079,0.006945,-0.01156,0.003805,0.002642,0.006448,-0.255979
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354,2018-01-30 15:25:00,0.000000,116650.0,116650.0,116657.522124,116680.681818,0.000000,0.000000,0.000199,-0.000075,...,-9.435696,-0.014536,0.000865,0.006079,0.006945,-0.01156,0.003805,0.002642,0.006448,-0.999970
355,2018-01-30 15:26:00,0.000000,116650.0,116650.0,116679.775281,116671.010460,0.000000,0.000000,-0.000075,-0.000194,...,-9.435696,-0.014536,0.000865,0.006079,0.006945,-0.01156,0.003805,0.002642,0.006448,0.000000
356,2018-01-30 15:27:00,-1.000000,116650.0,116650.0,116670.443205,116647.765630,0.000000,-0.000858,-0.000194,-0.000755,...,-9.435696,-0.014536,0.000865,0.006079,0.006945,-0.01156,0.003805,0.002642,0.006448,0.000000
357,2018-01-30 15:28:00,-1.000000,116650.0,116550.0,116647.765630,116559.743824,-0.000858,0.000000,-0.000755,0.000032,...,-9.434838,-0.014536,0.000865,0.006079,0.006945,-0.01156,0.003805,0.002642,0.006448,-0.999982


In [3]:
from chunk_pipeline import *
from prediction_ML_pipeline import save_dataframe_to_folder
import time
import os

logging.getLogger().setLevel(logging.WARNING)

## ---------ADJUST THIS---------------

ticker_lst = ['HII']


skip_training = True
year = "2018"
specific_date = "2018-01-30"
order_type_lst = ['combined']
predictive_lst = [True]
ret_type_lst = ["log_ret_ex"]
momentum_lst = [True]
file_name_lst= ["NOT_SAVING.csv"]

delta_lst = ['15S', '30S', '1min', '2min', '5min', '10min', '15min', '30min', '60min']

## ------------------------------------

# Check if files exist
full_results_df = None
folder_path = "/nfs/home/jingt/dissertation-iceberg/data/output_results/2018"
model_path = "/nfs/home/jingt/dissertation-iceberg/data/output_folder"
results_df_all = {i: [] for i in file_name_lst}

for ticker in ticker_lst:

    file_path_train = f'/nfs/home/jingt/dissertation-iceberg/data/training_data/_data_dwn_32_210_hidden_liquidity_{ticker}_2012-01-01_2012-12-31_1.7z'
    file_path_test = f"/nfs/data/lobster_data/lobster_raw/2017-19/_data_dwn_32_302__{ticker}_{year}-01-01_{year}-12-31_10.7z"

    if not os.path.exists(file_path_train):
        print(f"The file {file_path_train} does not exist.")
    if not os.path.exists(file_path_test):
        print(f"The file {file_path_test} does not exist.")
print("All files exist!")


for ticker in ticker_lst:

    model_name = f"xgboost_{ticker}.json"

    print(f"For ticker: {ticker}")
    start = time.time()
    archive_train_path = f"/nfs/home/jingt/dissertation-iceberg/data/training_data/_data_dwn_32_210_hidden_liquidity_{ticker}_2012-01-01_2012-12-31_1.7z"
    arhive_test_path = f"/nfs/data/lobster_data/lobster_raw/2017-19/_data_dwn_32_302__{ticker}_{year}-01-01_{year}-12-31_10.7z"


    params = {
        'objective': 'binary:logistic', 
        'learning_rate': 0.1,
        'max_depth': 6,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'n_estimators': 100,
        'eval_metric': 'logloss',  # Log loss for binary classification
    }

    if not skip_training:
        print("Model fitting started", flush=True)
        model, acc = process_and_train_xgb(archive_train_path, model_path, model_name, params=params)
        print("Model fitting completed!", flush=True)
    else:
        model = None
        acc = 0

    for order_type, file_name, predictive, ret_type, momentum in zip(order_type_lst, file_name_lst,
                                                                        predictive_lst, ret_type_lst,
                                                                        momentum_lst):
        print(f"For {order_type}, pred: {predictive}, ret_type: {ret_type}, momentum: {momentum}")
        print(f"{time.time() - start:.3f} seconds elapsed", flush=True)
        start = time.time()
        print("Order imbalance calculation started", flush=True)
        df_dict_2 = order_imbalance_calc(arhive_test_path, delta_lst=delta_lst, model=model,
                                    model_path=model_path, model_name=model_name, 
                                    order_type=order_type, specific_date=specific_date)

        print("Order imbalance calculation completed!", flush=True)

All files exist!
For ticker: HII
For combined, pred: True, ret_type: log_ret_ex, momentum: True
0.000 seconds elapsed
Order imbalance calculation started
Processed files: HII_2018-01-30_34200000_57600000_orderbook_10.csv HII_2018-01-30_34200000_57600000_message_10.csv
Order imbalance calculation completed!


In [4]:
params_df = pd.read_csv("/nfs/home/jingt/dissertation-iceberg/data/output_results/2018/all_log_ret_ex_2018_v2.csv")


In [5]:
ticker = 'AES'
ticker2 = 'HII'

delta = '1min'

params_df_ticker = params_df[(params_df['ticker'] == ticker) & (params_df['timeframe']== delta)]
params_df_ticker2 = params_df[(params_df['ticker'] == ticker2) & (params_df['timeframe']== delta)]


OI_df = df_dict[delta]
OI_df2 = df_dict_2[delta]

signal_df = OI_df[["datetime_bins", "fut_log_ret_ex"]]
signal_df['ticker'] = ticker

signal_df['signal'] = (float(params_df_ticker['intercept']) 
                    + float(params_df_ticker['params_vis']) * OI_df['order_imbalance_vis'] 
                    + float(params_df_ticker['params_hid']) * OI_df['order_imbalance_hid'])

signal_df2 = OI_df2[["datetime_bins", "fut_log_ret_ex"]]
signal_df2['ticker'] = ticker2

signal_df2['signal'] = (float(params_df_ticker2['intercept']) 
                    + float(params_df_ticker2['params_vis']) * OI_df2['order_imbalance_vis'] 
                    + float(params_df_ticker2['params_hid']) * OI_df2['order_imbalance_hid'])


In [6]:
OI_df2

,datetime_bins,order_imbalance_vis,first_midprice,last_midprice,first_weighted_mp,last_weighted_mp,log_ret,fut_log_ret,weighted_log_ret,fut_weighted_log_ret,...,fret_tClose,daily_ret,fut_daily_ret,fret_ClOp,fret_ClCl,daily_ret_ex,fut_daily_ret_ex,fret_ClOp_ex,fret_ClCl_ex,order_imbalance_hid
0,2018-01-30 09:31:00,1.0,2461000.0,2465750.0,2.468822e+06,2.470804e+06,0.001928,0.000304,0.000802,0.000167,...,-9.348193,-0.008745,-0.035808,0.005744,-0.030065,-0.005768,-0.032868,0.002307,-0.030561,0.000000
1,2018-01-30 09:32:00,0.0,2465750.0,2466500.0,2.470804e+06,2.471217e+06,0.000304,0.000122,0.000167,0.000067,...,-9.348477,-0.008745,-0.035808,0.005744,-0.030065,-0.005768,-0.032868,0.002307,-0.030561,0.000000
2,2018-01-30 09:33:00,0.0,2466450.0,2466750.0,2.471190e+06,2.471355e+06,0.000122,-0.001582,0.000067,-0.003561,...,-9.348599,-0.008745,-0.035808,0.005744,-0.030065,-0.005768,-0.032868,0.002307,-0.030561,0.000000
3,2018-01-30 09:34:00,0.0,2466750.0,2462850.0,2.473727e+06,2.464933e+06,-0.001582,0.001521,-0.003561,0.002505,...,-9.347098,-0.008745,-0.035808,0.005744,-0.030065,-0.005768,-0.032868,0.002307,-0.030561,0.000000
4,2018-01-30 09:35:00,0.0,2463050.0,2466800.0,2.465200e+06,2.471383e+06,0.001521,-0.001785,0.002505,-0.002329,...,-9.348619,-0.008745,-0.035808,0.005744,-0.030065,-0.005768,-0.032868,0.002307,-0.030561,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354,2018-01-30 15:25:00,0.0,2455200.0,2455150.0,2.456161e+06,2.456074e+06,-0.000020,0.000081,-0.000035,0.000040,...,-9.343905,-0.008745,-0.035808,0.005744,-0.030065,-0.005768,-0.032868,0.002307,-0.030561,0.000000
355,2018-01-30 15:26:00,0.0,2455200.0,2455400.0,2.456087e+06,2.456184e+06,0.000081,0.000489,0.000040,0.000173,...,-9.343987,-0.008745,-0.035808,0.005744,-0.030065,-0.005768,-0.032868,0.002307,-0.030561,0.000000
356,2018-01-30 15:27:00,1.0,2455400.0,2456600.0,2.456169e+06,2.456594e+06,0.000489,0.000000,0.000173,-0.000481,...,-9.344475,-0.008745,-0.035808,0.005744,-0.030065,-0.005768,-0.032868,0.002307,-0.030561,0.987281
357,2018-01-30 15:28:00,0.0,2456600.0,2456600.0,2.457776e+06,2.456594e+06,0.000000,0.000000,-0.000481,0.000000,...,-9.344475,-0.008745,-0.035808,0.005744,-0.030065,-0.005768,-0.032868,0.002307,-0.030561,-0.332675


In [7]:
import pandas as pd

In [8]:
signal_df_all = pd.concat([signal_df, signal_df2]).reset_index(drop=True)

In [9]:
percentile=0.5
top_percentile = int(percentile * 2)

In [10]:
signal_df_all['signal_rank'] = signal_df_all.groupby('datetime_bins')['signal'].rank(method='first', ascending=False)


In [11]:
signal_df_all['signal_rank'] = signal_df_all.groupby('datetime_bins')['signal'].rank(method='first', ascending=False)

top_signals = signal_df_all[signal_df_all['signal_rank'] <= top_percentile]
top_signals = top_signals[top_signals['signal'] > 0]
bottom_signals = signal_df_all[signal_df_all['signal_rank'] > top_percentile]
bottom_signals = bottom_signals[bottom_signals['signal'] < 0]

# Group by 'ticker' and count the occurrences for top signals
top_counts = top_signals.groupby('ticker').count()[['signal']]
top_counts.rename(columns={'signal': 'top_counts'}, inplace=True)

# Group by 'ticker' and count the occurrences for bottom signals
bottom_counts = bottom_signals.groupby('ticker').count()[['signal']]
bottom_counts.rename(columns={'signal': 'bottom_counts'}, inplace=True)

df_counts = pd.merge(top_counts, bottom_counts, left_index=True, right_index=True)

top_sum  = top_signals['fut_log_ret_ex'].sum()
bottom_sum = bottom_signals['fut_log_ret_ex'].sum()

result = top_sum - bottom_sum

In [12]:
top_signals

,datetime_bins,fut_log_ret_ex,ticker,signal,signal_rank
77,2018-01-30 10:48:00,0.001019,AES,0.000012,1.0
124,2018-01-30 11:35:00,0.000541,AES,0.000011,1.0
217,2018-01-30 13:08:00,-0.001318,AES,0.000003,1.0
236,2018-01-30 13:27:00,-0.001658,AES,0.000013,1.0
359,2018-01-30 09:31:00,-0.000970,HII,0.000005,1.0
...,...,...,...,...,...
712,2018-01-30 15:24:00,-0.000251,HII,0.000013,1.0
713,2018-01-30 15:25:00,0.000011,HII,0.000009,1.0
714,2018-01-30 15:26:00,0.000382,HII,0.000009,1.0
715,2018-01-30 15:27:00,0.000177,HII,0.000058,1.0


In [13]:
bottom_signals

,datetime_bins,fut_log_ret_ex,ticker,signal,signal_rank
0,2018-01-30 09:31:00,-0.008051,AES,-0.000017,2.0
1,2018-01-30 09:32:00,-0.004807,AES,-0.000037,2.0
2,2018-01-30 09:33:00,-0.000159,AES,-0.000025,2.0
3,2018-01-30 09:34:00,-0.000194,AES,-0.000030,2.0
4,2018-01-30 09:35:00,-0.002221,AES,-0.000033,2.0
...,...,...,...,...,...
561,2018-01-30 12:53:00,-0.000319,HII,-0.000038,2.0
576,2018-01-30 13:08:00,0.001260,HII,-0.000037,2.0
586,2018-01-30 13:18:00,0.000177,HII,-0.000042,2.0
636,2018-01-30 14:08:00,-0.000102,HII,-0.000029,2.0


In [14]:
import exchange_calendars as mcal

In [15]:
nasdaq = mcal.get_calendar('XNYS')  # XNYS is often used for NASDAQ

# Get the trading schedule for the entire year
trading_schedule = nasdaq.sessions_in_range(f'{year}-01-01', f'{year}-12-31')

# Convert to a list of strings
trading_days_lst = trading_schedule.strftime('%Y-%m-%d').tolist()

In [16]:
params_year = 2018

In [17]:
params_file_name = "all_log_ret_ex_2018_v2.csv"
params_folder_path = f"/nfs/home/jingt/dissertation-iceberg/data/output_results/{params_year}"

params_file_path = os.path.join(params_folder_path, params_file_name)
params_df = pd.read_csv(params_file_path)

In [18]:
'AES' in params_df['ticker'].unique()

True

In [19]:
params_df['ticker'].unique()

array(['AES', 'ALB', 'AOS', 'APA', 'BEN', 'BXP', 'CPB', 'DVA', 'FFIV',
       'FRT', 'HII', 'HRL', 'HSIC', 'INCY', 'MHK', 'NWSA', 'PNW', 'RL',
       'TAP', 'WYNN'], dtype=object)

In [20]:
positive_sum = signal_df[signal_df['signal'] > 0]['fut_log_ret_ex'].sum()
# Calculate the sum of fut_log_ret_ex for negative signals
negative_sum = signal_df[signal_df['signal'] < 0]['fut_log_ret_ex'].sum()

In [21]:
result = positive_sum - negative_sum

In [22]:
result_lst = [result]

In [23]:
nasdaq = mcal.get_calendar('XNYS')  # XNYS is often used for NASDAQ

# Get the trading schedule for the entire year
trading_schedule = nasdaq.sessions_in_range(f'{year}-01-01', f'{year}-12-31')

# Convert to a list of strings
trading_days_lst = trading_schedule.strftime('%Y-%m-%d').tolist()

In [24]:
trading_days_lst

['2018-01-02',
 '2018-01-03',
 '2018-01-04',
 '2018-01-05',
 '2018-01-08',
 '2018-01-09',
 '2018-01-10',
 '2018-01-11',
 '2018-01-12',
 '2018-01-16',
 '2018-01-17',
 '2018-01-18',
 '2018-01-19',
 '2018-01-22',
 '2018-01-23',
 '2018-01-24',
 '2018-01-25',
 '2018-01-26',
 '2018-01-29',
 '2018-01-30',
 '2018-01-31',
 '2018-02-01',
 '2018-02-02',
 '2018-02-05',
 '2018-02-06',
 '2018-02-07',
 '2018-02-08',
 '2018-02-09',
 '2018-02-12',
 '2018-02-13',
 '2018-02-14',
 '2018-02-15',
 '2018-02-16',
 '2018-02-20',
 '2018-02-21',
 '2018-02-22',
 '2018-02-23',
 '2018-02-26',
 '2018-02-27',
 '2018-02-28',
 '2018-03-01',
 '2018-03-02',
 '2018-03-05',
 '2018-03-06',
 '2018-03-07',
 '2018-03-08',
 '2018-03-09',
 '2018-03-12',
 '2018-03-13',
 '2018-03-14',
 '2018-03-15',
 '2018-03-16',
 '2018-03-19',
 '2018-03-20',
 '2018-03-21',
 '2018-03-22',
 '2018-03-23',
 '2018-03-26',
 '2018-03-27',
 '2018-03-28',
 '2018-03-29',
 '2018-04-02',
 '2018-04-03',
 '2018-04-04',
 '2018-04-05',
 '2018-04-06',
 '2018-04-

In [25]:
test_lst = list(range(251))

In [26]:

df = pd.DataFrame([test_lst], columns=trading_days_lst)

In [27]:
params_df[params_df['ticker'] == 'HRL']

,timeframe,intercept,adj_R2,params_vis,tvals_vis,params_hid,tvals_hid,ticker,model_acc
77,30S,-0.000003,1.0,0.000011,4.947912,0.000007,1.538054,HRL,0
78,1min,-0.000006,1.0,0.000013,3.287561,-0.000009,-1.302902,HRL,0
79,2min,-0.000014,1.0,0.000022,2.477267,0.000026,2.572380,HRL,0
80,5min,0.000007,1.0,0.000080,2.993986,-0.000009,-0.485231,HRL,0
81,10min,0.000106,1.0,0.000054,0.779817,-0.000019,-0.501877,HRL,0
82,15min,0.000192,1.0,0.000038,0.334619,-0.000017,-0.296727,HRL,0
83,30min,0.000284,1.0,0.000087,0.331542,-0.000029,-0.226071,HRL,0


In [28]:
folder_path = "/nfs/home/jingt/dissertation-iceberg/data/output_results/2018"

In [29]:
df_params = pd.read_csv("/nfs/home/jingt/dissertation-iceberg/data/output_results/2018/all_log_ret_ex_2018_v2.csv")

In [30]:
df_params.groupby('timeframe').apply(lambda x: (x['tvals_hid'].abs() > 1.64).sum())

timeframe
10min     3
15min     1
1min      8
2min      8
30S      12
30min     0
5min      5
dtype: int64

In [31]:
df_params.groupby('timeframe').apply(lambda x: (x['tvals_vis'].abs() > 1.64).sum())

timeframe
10min     0
15min     0
1min     10
2min      7
30S      16
30min     0
5min      6
dtype: int64

In [32]:
import pandas as pd
df_strat = pd.read_csv("/nfs/home/jingt/dissertation-iceberg/data/output_results/strategy_results/all_strat_single_comb_log_ret_ex.csv")

In [33]:
df_strat['final_PnL'].sum()

-1.074481758987746

In [34]:
OI_df

,datetime_bins,order_imbalance_vis,first_midprice,last_midprice,first_weighted_mp,last_weighted_mp,log_ret,fut_log_ret,weighted_log_ret,fut_weighted_log_ret,...,fret_tClose,daily_ret,fut_daily_ret,fret_ClOp,fret_ClCl,daily_ret_ex,fut_daily_ret_ex,fret_ClOp_ex,fret_ClCl_ex,order_imbalance_hid
0,2018-01-30 09:31:00,0.249479,117950.0,118450.0,117950.000000,118410.263022,0.004230,-0.006777,0.003895,-0.006335,...,-9.451009,-0.014536,0.000865,0.006079,0.006945,-0.01156,0.003805,0.002642,0.006448,-0.187515
1,2018-01-30 09:32:00,-0.291682,118450.0,117650.0,118410.263022,117662.500000,-0.006777,-0.004259,-0.006335,-0.004462,...,-9.444232,-0.014536,0.000865,0.006079,0.006945,-0.01156,0.003805,0.002642,0.006448,-0.836726
2,2018-01-30 09:33:00,-0.437851,117650.0,117150.0,117657.142857,117133.333333,-0.004259,0.000000,-0.004462,0.000000,...,-9.439973,-0.014536,0.000865,0.006079,0.006945,-0.01156,0.003805,0.002642,0.006448,0.556370
3,2018-01-30 09:34:00,-0.357258,117150.0,117150.0,117150.000000,117150.000000,0.000000,0.000000,0.000000,0.000000,...,-9.439973,-0.014536,0.000865,0.006079,0.006945,-0.01156,0.003805,0.002642,0.006448,0.000000
4,2018-01-30 09:35:00,-0.375000,117150.0,117150.0,117150.000000,117150.000000,0.000000,-0.001709,0.000000,-0.001908,...,-9.439973,-0.014536,0.000865,0.006079,0.006945,-0.01156,0.003805,0.002642,0.006448,-0.255979
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354,2018-01-30 15:25:00,0.000000,116650.0,116650.0,116657.522124,116680.681818,0.000000,0.000000,0.000199,-0.000075,...,-9.435696,-0.014536,0.000865,0.006079,0.006945,-0.01156,0.003805,0.002642,0.006448,-0.999970
355,2018-01-30 15:26:00,0.000000,116650.0,116650.0,116679.775281,116671.010460,0.000000,0.000000,-0.000075,-0.000194,...,-9.435696,-0.014536,0.000865,0.006079,0.006945,-0.01156,0.003805,0.002642,0.006448,0.000000
356,2018-01-30 15:27:00,-1.000000,116650.0,116650.0,116670.443205,116647.765630,0.000000,-0.000858,-0.000194,-0.000755,...,-9.435696,-0.014536,0.000865,0.006079,0.006945,-0.01156,0.003805,0.002642,0.006448,0.000000
357,2018-01-30 15:28:00,-1.000000,116650.0,116550.0,116647.765630,116559.743824,-0.000858,0.000000,-0.000755,0.000032,...,-9.434838,-0.014536,0.000865,0.006079,0.006945,-0.01156,0.003805,0.002642,0.006448,-0.999982


In [4]:
import pandas as pd

df_results = pd.read_csv("/nfs/home/jingt/dissertation-iceberg/data/output_results/2018/all_log_ret_ex_iceberg_momentum_2018_v2.csv")

In [1]:

df_results.groupby("timeframe").apply(lambda x: (x['tvals_hid'].abs() > 1.64).sum())

NameError: name 'df_results' is not defined

In [5]:
import pandas as pd
delta='1min'
start_date = "2019-01-01"
end_date = "2019-01-01"

# Set the time to start at 09:30 and end at 15:30
start_datetime = pd.Timestamp.combine(pd.to_datetime(start_date), pd.Timestamp("09:30").time())
start_datetime = start_datetime + pd.Timedelta(delta)
end_datetime = pd.Timestamp.combine(pd.to_datetime(end_date), pd.Timestamp("15:30").time())

# Generate the full range of datetime bins with the specified frequency
full_range = pd.date_range(start=start_datetime, end=end_datetime, freq=delta)

# Create the DataFrame with the datetime bins
full_bins = pd.DataFrame(full_range, columns=['datetime_bins']) 

NameError: name 'delta' is not defined